In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class Simulator:
    def __init__(
        self, 
        num_genes = 300000, 
        avg_num_sgrnas = 5, 
        num_treatment = 2, 
        num_control = 2, 
        #ask about reads -- does this mean that the total numbers in the library should sum to this number 
        total_reads_treatment = 1000,
        total_reads_control = 1000,
        total_NTCs = 1000,
        fraction_enriched = 0.2,
        fraction_depleted = 0.2,
        fraction_NTC = 0.2):
        
        #use the inputted fractions to define the normal one? Ensure total fractions = 1
        
        self.num_genes = num_genes
        self.avg_num_sgrnas = avg_num_sgrnas
        self.num_treatment = num_treatment
        self.num_control = num_control
        self.total_reads_treatment = total_reads_treatment
        self.total_reads_control = total_reads_control
        self.total_NTCs = total_NTCs
        self.fraction_enriched = fraction_enriched
        self.fraction_depleted = fraction_depleted
        self.fraction_NTC = fraction_NTC
        
    
    
    def sample(self):
        df = pd.DataFrame({"gene": ["gene" + str(i) for i in np.arange(self.num_genes)]})
        
        for i in np.arange(self.num_treatment):
            df["lib_treatment_" + str(i+1)] = np.random.randint(1, 10, size=self.num_genes)
            
        for i in np.arange(self.num_control):
            df["lib_control_" + str(i+1)] = np.random.randint(1, 10, size=self.num_genes)
        
        return df

In [3]:
trial = Simulator(num_genes=10000, num_treatment=3)
print(trial.sample())

          gene  lib_treatment_1  lib_treatment_2  lib_treatment_3  \
0        gene0                3                5                5   
1        gene1                9                3                2   
2        gene2                1                9                1   
3        gene3                4                6                6   
4        gene4                4                4                3   
...        ...              ...              ...              ...   
9995  gene9995                3                4                7   
9996  gene9996                6                3                8   
9997  gene9997                8                9                8   
9998  gene9998                9                2                9   
9999  gene9999                7                2                9   

      lib_control_1  lib_control_2  
0                 7              3  
1                 1              8  
2                 5              7  
3                 1    